In [1]:
import torch
import numpy as np
import time
import os
from utils import HDDataset
from model import BModel
from torchvision import datasets, transforms
import scipy
from encoder import RandomFourierEncoder
import main2_fmnist 
import importlib
importlib.reload(main2_fmnist)
import csv
from scipy.io import savemat, loadmat
from torch.utils.data import DataLoader, Subset, TensorDataset


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
gorder = 2
dim = 10000
gamma = 0.3
lr = 0.01
classes = 10
channels = 3
epochs = 3
train = main2_fmnist.train
test = main2_fmnist.test

In [4]:
@torch.no_grad()                       # no autograd graph needed
def flip_rows_(tensor: torch.Tensor, perc: float, *, inplace: bool = True) -> torch.Tensor:
    """
    Flip the sign of `perc` fraction of elements **in every row** of `tensor`.

    Args
    ----
    tensor : torch.Tensor               # shape (N, D)
        2-D tensor whose rows will be sign-flipped.
    perc   : float                      # 0 ≤ perc ≤ 1
        Fraction of positions per row to flip.
    inplace: bool (default True)
        If True, modify `tensor` in place and return it;
        otherwise return a flipped clone.

    Returns
    -------
    torch.Tensor
        The tensor with sign-flipped rows (same object if `inplace` is True).
    """
    if not (0.0 <= perc <= 1.0):
        raise ValueError("`perc` must be in the interval [0, 1].")

    # Short-circuit trivial cases
    if perc == 0.0 or tensor.numel() == 0:
        return tensor
    if not inplace:
        tensor = tensor.clone()

    N, D = tensor.shape
    k = int(round(D * perc))            # exact count per row
    if k == 0:
        return tensor                   # nothing to flip

    device = tensor.device
    for r in range(N):
        # choose k unique positions in this row
        idx = torch.randperm(D, device=device)[:k]
        tensor[r, idx] *= -1            # in-place sign change

    return tensor


In [5]:
trainset = datasets.FashionMNIST(root='../../Data', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.FashionMNIST(root='../../Data', train=False, download=True, transform=transforms.ToTensor())
n_splits = 20
split_size = len(testset) // n_splits
flip_percs = np.arange(0.0, 0.51, 0.05)
accuracies1 = np.zeros((len(flip_percs), n_splits))
hyperdim = 25_000
input_dim = torch.prod(torch.tensor(list(trainset[0][0].size())))
print(input_dim)
for i, perc in enumerate(flip_percs):
    print(f'Flipping Percentage: {np.round(perc, 2)} ------------')
    encoder = RandomFourierEncoder(input_dim, gamma, gorder, output_dim=hyperdim)
    encoder.build_item_mem()
    train_hd, y_train = encoder.encode_data_extract_labels(trainset)
    train_dataset = HDDataset(train_hd, y_train)
    trainloader = DataLoader(train_dataset, batch_size=1024, shuffle=True, pin_memory=True, num_workers=2)
    for split_idx in range(n_splits):
        # Test
        print(f'Split Index: {split_idx+1}')
        start_idx = split_idx * split_size
        end_idx = start_idx + split_size
        split_subset = Subset(testset, range(start_idx, end_idx))
        test_hd, y_test = encoder.encode_data_extract_labels(split_subset)
        flip_rows_(test_hd, perc=perc)  
        test_dataset = HDDataset(test_hd, y_test)
        testloader = DataLoader(test_dataset, batch_size=512, shuffle=False, pin_memory=True, num_workers=2)
        accuracies1[i, split_idx] = train(trainloader, testloader, lr, hyperdim)
        del testloader
        del test_dataset
        del test_hd
        
    del train_hd
    del trainloader
    del train_dataset
    print(f'Average Accuracy for Flipping Percentage={np.round(perc, 2)}: {np.mean(accuracies1[i, :])}')

tensor(784)
Flipping Percentage: 0.0 ------------
the threshold to discretize fourier features to group elements tensor([0.])
Split Index: 1
Split Index: 2
Split Index: 3
Split Index: 4
Split Index: 5
Split Index: 6
Split Index: 7
Split Index: 8
Split Index: 9
Split Index: 10
Split Index: 11
Split Index: 12
Split Index: 13
Split Index: 14
Split Index: 15
Split Index: 16
Split Index: 17
Split Index: 18
Split Index: 19
Split Index: 20
Average Accuracy for Flipping Percentage=0.0: 84.73
Flipping Percentage: 0.05 ------------
the threshold to discretize fourier features to group elements tensor([0.])
Split Index: 1
Split Index: 2
Split Index: 3
Split Index: 4
Split Index: 5
Split Index: 6
Split Index: 7
Split Index: 8
Split Index: 9
Split Index: 10
Split Index: 11
Split Index: 12
Split Index: 13
Split Index: 14
Split Index: 15
Split Index: 16
Split Index: 17
Split Index: 18
Split Index: 19
Split Index: 20
Average Accuracy for Flipping Percentage=0.05: 83.53
Flipping Percentage: 0.1 -------

In [6]:
np.mean(accuracies1, axis=1)

array([84.73, 83.53, 80.99, 77.33, 75.  , 68.03, 59.16, 41.35, 35.69,
       20.06,  9.17])

In [7]:
savemat('RFFHDC_FashionMNIST.mat', {'RFFHDC_FashionMNIST': accuracies1})